In [1]:
%load_ext autoreload

In [2]:
import numpy as np
from functools import partial
import visualizations_utils as viz_utils
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.solvers import mathematicalprogram as mp
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint

import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.all import RationalForwardKinematics, WriteCspacePolytopeToFile, ReadCspacePolytopeFromFile
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid

# Build and set up the visualization the plant and the visualization of the C-space obstacle

Note that running this cell multiple times will establish multiple meshcat instances which can fill up your memory. It is a good idea to call "pkill -f meshcat" from the command line before re-running this cell


In [4]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
oneDOF_iiwa_asset = FindResourceOrThrow("drake/C_Iris_Examples/assets/oneDOF_iiwa7_with_box_collision.sdf")
twoDOF_iiwa_asset = FindResourceOrThrow("drake/C_Iris_Examples/assets/twoDOF_iiwa7_with_box_collision.sdf")

box_asset = FindResourceOrThrow("drake/C_Iris_Examples/assets/box_small.urdf")

models = []
models.append(parser.AddModelFromFile(box_asset))
models.append(parser.AddModelFromFile(twoDOF_iiwa_asset))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset))



locs = [[0.,0.,0.],
        [0.,.55,0.],
        [0.,-.55,0.]]
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("base", models[0]),
                 RigidTransform(locs[0]))
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("iiwa_twoDOF_link_0", models[1]), 
                 RigidTransform(RollPitchYaw([0,0, -np.pi/2]).ToRotationMatrix(), locs[1]))
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("iiwa_oneDOF_link_0", models[2]), 
                 RigidTransform(RollPitchYaw([0,0, -np.pi/2]).ToRotationMatrix(), locs[2]))


plant.Finalize()

idx = 0
q0 = [0.0, 0.0, 0.0]
q_low  = [-1.7, -2., -1.7]
q_high = [ 1.7,  2.,  1.7]
# set the joint limits of the plant
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            joint.set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([q_high[idx]]))
            idx += 1
        
# construct the RationalForwardKinematics of this plant. This object handles the
# computations for the forward kinematics in the tangent-configuration space
Ratfk = RationalForwardKinematics(plant)

# the point about which we will take the stereographic projections
q_star = np.zeros(3)

#compute limits in t-space
limits_t = []
for q in [q_low, q_high]:
    limits_t.append(Ratfk.ComputeTValue(np.array(q), q_star))

do_viz = True

# This line builds the visualization. Change the viz_role to Role.kIllustration if you
# want to see the plant with its illustrated geometry or to Role.kProximity
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, viz_role=Role.kIllustration)
diagram = visualizer.diagram

# This line will run marching cubes to generate a mesh of the C-space obstacle
# Increase N to increase the resolution of the C-space obstacle.
visualizer.visualize_collision_constraint(N = 30)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7014/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7015/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6014...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7014/static/
Connected to meshcat-server.


## Set up the sliders so we can move the plant around manually

In [5]:
sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    visualizer.showres(q)
    visualizer.visualize_planes()
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

visualizer.jupyter_cell()

FloatSlider(value=0.0, description='q0', max=1.7, min=-1.7)

FloatSlider(value=0.0, description='q1', max=2.0, min=-2.0)

FloatSlider(value=0.0, description='q2', max=1.7, min=-1.7)

In [6]:
# filter fused joints self collisions so they don't interfere with collision engine
digaram = visualizer.diagram
context = visualizer.diagram_context
sg_context = scene_graph.GetMyContextFromRoot(context)
inspector = scene_graph.model_inspector()

pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()

gids = [gid for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)]
get_name_of_gid = lambda gid : inspector.GetName(gid)
gids.sort(key=get_name_of_gid)
iiwa_oneDOF_gids = [gid for gid in gids if "iiwa7_oneDOF::" in get_name_of_gid(gid)]
iiwa_twoDOF_gids = [gid for gid in gids if "iiwa7_twoDOF::" in get_name_of_gid(gid)]

oneDOF_fused_col_geom = iiwa_oneDOF_gids[2:]
iiwa_oneDOF_fused_set = GeometrySet(oneDOF_fused_col_geom)
twoDOF_fused_col_geom = iiwa_twoDOF_gids[4:]
iiwa_twoDOF_fused_set = GeometrySet(twoDOF_fused_col_geom)
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(iiwa_oneDOF_fused_set))
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(iiwa_twoDOF_fused_set))

# Setup IRIS Options and Generate Regions

In [7]:
# Some examples of some seed points which give large regions using the
# non-linear programming approach to IRIS described in Algorithm 3


seed_points_q = np.array([
                        [0.8, 1.3, -0.8],  # START: blue low green up
                        [0.1, 0.9, -1.2],     # GOAL: green low other up
                        [0.5, -1.9, -0.9],
                        [1.3, 1.7, -1.7],
                        [1.6, -0.3, -1.2], 
                        [-0.2,-1.5,-1.4],
                         ]) 

seed_points = np.array([Ratfk.ComputeTValue(seed_points_q[idx], np.zeros((3,)))\
                        for idx in range(seed_points_q.shape[0])])
if do_viz:
    visualizer.plot_seedpoints(seed_points)


In [8]:
regions = []
ellipses = []

iris_options = IrisOptionsRationalSpace()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 20
iris_options.configuration_space_margin = 1e-5
iris_options.max_faces_per_collision_pair = 60
iris_options.termination_threshold = -1
iris_options.q_star = np.zeros(3)
iris_options.relative_termination_threshold = 0.05
iris_options.enable_ibex = False

for i, s in enumerate(seed_points):
    q = Ratfk.ComputeQValue(s, np.zeros((3,)))
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context), q)
    r = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context), iris_options)
    regions.append(r)
    ellipses.append(r.MaximumVolumeInscribedEllipsoid())
if do_viz:
    visualizer.plot_regions(regions,
                            ellipses=ellipses,
                            region_suffix='_original',
                            randomize_colors = True)

# Certify one region

In [9]:
cspace_free_region_certifier = rational_forward_kinematics.CspaceFreeRegion(diagram, plant, scene_graph,
                                   rational_forward_kinematics.SeparatingPlaneOrder.kAffine,
                                   rational_forward_kinematics.CspaceRegionType.kGenericPolytope)
filtered_collision_pairs = set()
solver_options = mp.SolverOptions()
# make the solver verbose
solver_options.SetOption(mp.CommonSolverOption.kPrintToConsole, 1)

## first search for a minimal, uniform contraction of a proposed region which can be certified

In [10]:
bisection_contraction_regions = []
for i in range(len(seed_points)):
    region_to_certify = regions[i]
    seed_point = seed_points[i,:]
    if do_viz:
        visualizer.plot_regions([region_to_certify], ellipses=None, region_suffix='_original_to_certify')

    binary_search_options = rational_forward_kinematics.BinarySearchOption()
    binary_search_options.epsilon_max = 0 # it is very unlikely that we can find a uniform expansion of the current region
    binary_search_options.max_iters = 15
    # speed up the bisection search by taking non-uniform steps when possible
    binary_search_options.search_d = True
    # find the smallest e such that At <= b + e1 still contains our seed point.
    binary_search_options.epsilon_min = rational_forward_kinematics.FindEpsilonLower(region_to_certify.A(), region_to_certify.b(),
                                                                                     limits_t[0], limits_t[1],
                                                                                     seed_point)
    #use as many threads as possible to speed up computation
    binary_search_options.num_threads = -1
    try:
        certified_region_contraction_solution = cspace_free_region_certifier.CspacePolytopeBinarySearch(q_star,
                                                                         filtered_collision_pairs,
                                                                         region_to_certify.A(),
                                                                         region_to_certify.b(),
                                                                         binary_search_options, 
                                                                         solver_options,
                                                                         seed_point)
        certified_region_contraction = HPolyhedron(certified_region_contraction_solution.C,
                                                   certified_region_contraction_solution.d)
        bisection_contraction_regions.append(certified_region_contraction_solution)
    #     if do_viz:
    #         visualizer.plot_regions([certified_region_contraction], ellipses=None,
    #                                 region_suffix='_certified_region_contraction',
    #                                 randomize_colors = True)
        WriteCspacePolytopeToFile(certified_region_contraction_solution, plant, inspector, f"pinball_iiwa_regions/bisection_contraction_regions/solution{i}.txt", 10)
    except:
        pass

(box_scene::Box1, iiwa7_twoDOF::iiwa_twoDOF_link_7_collision)
(box_scene::Box1, iiwa7_twoDOF::iiwa_twoDOF_link_7_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_2_collision, iiwa7_twoDOF::iiwa_twoDOF_link_4_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_4_collision, iiwa7_oneDOF::iiwa_oneDOF_link_6_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_4_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_5_collision, iiwa7_oneDOF::iiwa_oneDOF_link_6_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_5_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_6_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)

INFO:drake:Found Lagrangian multiplier and separating planes
INFO:drake:max(power(det(P), 1/4))=0.41129876781969027, solver_time 0.028411149978637695
INFO:drake:search d is successful = true
INFO:drake:max(power(det(P), 1/4))=0.4132983739738706, solver_time 0.02214789390563965
INFO:drake:Found Lagrangian multiplier and separating planes
IN

(box_scene::Box1, iiwa7_twoDOF::iiwa_twoDOF_link_7_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_5_collision, iiwa7_oneDOF::iiwa_oneDOF_link_4_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_5_collision, iiwa7_oneDOF::iiwa_oneDOF_link_5_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_6_collision, iiwa7_oneDOF::iiwa_oneDOF_link_3_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_6_collision, iiwa7_oneDOF::iiwa_oneDOF_link_4_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_6_collision, iiwa7_oneDOF::iiwa_oneDOF_link_5_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_7_collision, iiwa7_oneDOF::iiwa_oneDOF_link_3_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_7_collision, iiwa7_oneDOF::iiwa_oneDOF_link_4_collision)

(iiwa7_twoDOF::iiwa_twoDOF_link_6_collision, iiwa7_oneDOF::iiwa_oneDOF_link_3_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_6_collision, iiwa7_oneDOF::iiwa_oneDOF_link_4_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_7_collision, iiwa7_oneDOF::iiwa_oneDOF_link_3_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_7_collision, ii

In [14]:
len(bisection_contraction_regions)

2

## Now that we have a certified region, we can grow it using bilinear alternations

In [12]:
bilinear_alternation_regions = []
for sol in bisection_contraction_regions:
    bilinear_alternation_options = rational_forward_kinematics.BilinearAlternationOption()
    bilinear_alternation_options.max_iters = 20
    bilinear_alternation_options.lagrangian_backoff_scale = 0
    bilinear_alternation_options.polytope_backoff_scale = 0
    # number of threads used to solve this optimization.
    bilinear_alternation_options.num_threads = -1

    certified_region_final_solution, polytope_volumes, ellipsoid_determinants = cspace_free_region_certifier.CspacePolytopeBilinearAlternation(q_star,
                                                                     filtered_collision_pairs,
                                                                     certified_region_contraction.A(),
                                                                     # it is helpful to pullback the solution of the previous step a little bit
                                                                     certified_region_contraction.b()-5e-2,
                                                                     bilinear_alternation_options, 
                                                                     solver_options,
                                                                     seed_point)
    final_certified_region = HPolyhedron(certified_region_final_solution.C, certified_region_final_solution.d)
#     if do_viz:
#         visualizer.plot_regions([final_certified_region], ellipses=None,
#                                 region_suffix='_certified_region_final',
#                                 randomize_colors = True)
    bilinear_alternation_regions.append(final_certified_region)
    WriteCspacePolytopeToFile(certified_region_contraction_solution, plant, inspector, f"pinball_iiwa_regions/bilinear_alternation_regions/solution{i}.txt", 10)

INFO:drake:Found Lagrangian multiplier and separating planes
INFO:drake:Lagrangian step time 5.822 s
INFO:drake:max(power(det(P), 1/4))=0.44486269102073006, solver_time 0.02176499366760254
INFO:drake:cost improvement inf
INFO:drake:mosek info 0, UnknownError
INFO:drake:Found Lagrangian multiplier and separating planes
INFO:drake:Lagrangian step time 5.924 s
INFO:drake:max(power(det(P), 1/4))=0.44486269102073006, solver_time 0.01642298698425293
INFO:drake:cost improvement inf
INFO:drake:mosek info 0, UnknownError


## Visualizing the separating planes
Our certificate of non-collision for a given region contains one parametric, separating hyperplane per collision pair. In this scene, there are 137 pairs of objects which could collide. We plot two planes of interest. After running this cell, you can move the plant with the slider and see how the planes move as function of the configuration.

In [ ]:
oneDOF_iiwa_end_gid = iiwa_oneDOF_gids[-1]
twoDOF_iiwa_end_gid = iiwa_twoDOF_gids[-1]
twoDOF_iiwa_base_gid = iiwa_twoDOF_gids[0]
id_pairs_of_interest = [(oneDOF_iiwa_end_gid, twoDOF_iiwa_end_gid),
                        (oneDOF_iiwa_end_gid, twoDOF_iiwa_base_gid)]

            
visualizer.collision_pairs_of_interest = id_pairs_of_interest
visualizer.certified_region_solution_list = [certified_region_contraction_solution]